# 8 Plots

On here we should have several plots we will create to be able to present our analysis in such a way that it can be very well presented and appear nicely in our report and our 

### Chart Ideas so far
    - rediction vs Actual curves

    - Histogram of RUL errors

    - Distribution of lifespans (already in preprocessing)

    - Comparison bar chart (RMSE/MAE per model)

    - Optional: Training/Validation loss over epochs for DL models

In [1]:
# plots.py (visualisation functions for RUL prediction)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import display


def plot_actual_vs_predicted(y_true, y_pred, model_name="Model", dataset_name="FD001"):
    plt.figure(figsize=(10, 6))
    plt.plot(y_true, label="Actual RUL", linestyle='dashed', marker='o')
    plt.plot(y_pred, label="Predicted RUL", linestyle='dashed', marker='x')
    plt.title(f"Actual vs Predicted RUL – {model_name} ({dataset_name})")
    plt.xlabel("Sample Index")
    plt.ylabel("Remaining Useful Life")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_loss_curve(history, model_name="Model", dataset_name="FD001"):
    if not hasattr(history, "history"):
        print("⚠️ Invalid history object.")
        return

    plt.figure(figsize=(8, 5))
    plt.plot(history.history["loss"], label="Training Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title(f"Training vs Validation Loss – {model_name} ({dataset_name})")
    plt.xlabel("Epochs")
    plt.ylabel("Loss (MSE)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_metric_comparison(metrics_list, dataset_name="FD001"):
    """
    metrics_list: list of dicts from evaluate_model(), e.g.,
        [{"model": "LSTM", "RMSE": 15.2, "MAE": 11.3}, ...]
    """
    df = pd.DataFrame(metrics_list)
    df = df.set_index("model")

    ax = df.plot(kind="bar", figsize=(10, 6))
    plt.title(f"RMSE / MAE Comparison – {dataset_name}")
    plt.ylabel("Error")
    plt.xticks(rotation=0)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_error_histogram(y_true, y_pred, model_name="Model", dataset_name="FD001"):
    errors = y_pred - y_true
    plt.figure(figsize=(8, 5))
    plt.hist(errors, bins=30, edgecolor='black')
    plt.title(f"Prediction Error Distribution – {model_name} ({dataset_name})")
    plt.xlabel("Prediction Error (Predicted - Actual)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def show_prediction_table(y_true, y_pred, model_name="Model", n=10):
    df = pd.DataFrame({
        "Actual RUL": y_true,
        "Predicted RUL": y_pred,
        "Error": y_pred - y_true
    })

    df_sample = df.head(n).copy()
    df_sample.index = [f"Engine {i+1}" for i in range(n)]
    display(df_sample.round(2))


# ✨ Additional Plot Ideas to Consider Later

def plot_rul_boxplot(y_true, y_pred, model_name="Model"):
    errors = y_pred - y_true
    df = pd.DataFrame({"Errors": errors, "Type": model_name})
    sns.boxplot(data=df, x="Type", y="Errors")
    plt.title("Boxplot of Prediction Errors")
    plt.ylabel("Prediction Error")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def plot_rul_by_engine(y_true, y_pred, unit_ids=None, model_name="Model"):
    df = pd.DataFrame({"Engine": unit_ids, "Actual": y_true, "Predicted": y_pred})
    df_melted = df.melt(id_vars=["Engine"], value_vars=["Actual", "Predicted"],
                        var_name="Type", value_name="RUL")
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_melted, x="Engine", y="RUL", hue="Type")
    plt.title(f"Predicted vs Actual RUL by Engine – {model_name}")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def plot_cross_dataset_heatmap(results_dict, metric='RMSE'):
    """
    Visualises generalisation capability via heatmap.

    Parameters:
        results_dict (dict): 
            {
                'FD001': {'FD001': 12.3, 'FD002': 20.1, ...},
                'FD002': {'FD001': 15.0, 'FD002': 10.8, ...},
                ...
            }
        metric (str): 'RMSE' or 'MAE' (for title only)
    """
    df = pd.DataFrame(results_dict)
    plt.figure(figsize=(8, 6))
    sns.heatmap(df, annot=True, fmt=".1f", cmap="YlGnBu")
    plt.title(f"{metric} – Train vs Test Dataset")
    plt.xlabel("Test Dataset")
    plt.ylabel("Train Dataset")
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_cross_dataset_bars(results_dict, metric='RMSE'):
    """
    Creates a grouped bar chart for train-test RMSE or MAE.
    """
    df = pd.DataFrame(results_dict).T.reset_index().melt(id_vars='index', 
                      var_name='Test Set', value_name=metric)
    df.rename(columns={'index': 'Train Set'}, inplace=True)

    plt.figure(figsize=(10, 6))
    sns.barplot(data=df, x='Train Set', y=metric, hue='Test Set')
    plt.title(f"{metric} by Train/Test Combination")
    plt.ylabel(metric)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
